# Using `pyologger` data processing pipeline with `DiveDB`
Uses classes `Metadata` and `DataReader` to facilitate data intake, processing, and alignment. 

## Read deployment metadata

In [ ]:
# Import libraries and set working directory (adjust to fit your preferences)
import os
import pickle
from pyologger.load_data.datareader import DataReader
from pyologger.load_data.metadata import Metadata
from pyologger.plot_data.plotter import *
from pyologger.calibrate_data.calibrate_acc_mag import *


# Change the current working directory to the root directory
# os.chdir("/Users/fbar/Documents/GitHub/pyologger")
os.chdir("/Users/jessiekb/Documents/GitHub/pyologger")

root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")

# Verify the current working directory
print(f"Current working directory: {root_dir}")

In [ ]:
import xarray as xr
#import netcdf4

# Load the NetCDF file
file_path = f"{data_dir}/2004001_TrackTDR_RawCurated.nc"
dataset = xr.open_dataset(file_path)

# Print the dataset information
print(dataset)

In [ ]:
# Initialize the info class
metadata = Metadata()
metadata.fetch_databases(verbose=False)

# Save databases
dep_db = metadata.get_metadata("dep_DB")
logger_db = metadata.get_metadata("logger_DB")
rec_db = metadata.get_metadata("rec_DB")
animal_db = metadata.get_metadata("animal_DB")

In [ ]:
# Check out the deployment dataframe.
dep_db_sorted = dep_db.sort_values('Rec Date')
dep_db_sorted

## Read files in deployment folder

Steps for Processing Deployment Data:

1. **Select Deployment Folder**:
   - **Description:** Asks the user for input to select a deployment folder to kick off the data reading process. In your folder name, you can have any suffix after Deployment ID. It will check and stop if there are two that fit.
   - **Function Used:** `check_deployment_folder()`

2. **Initialize Deployment Folder**:
   - **Description:** Starts the main `read_files` process with the selected deployment folder.
   - **Function Used:** `read_files()`

3. **Fetch Metadata**:
   - **Description:** Retrieve necessary data from the metadata database, including logger information.
   - **Function Used:** `metadata.fetch_databases()`

4. **Organize Files by Logger ID**:
   - **Description:** Group files by logger ID for processing.
   - **Function Used:** `read_files()` (This is the main function)

5. **Check for Existing Processed Files**:
   - **Description:** Verify if the outputs folder already contains processed files for each logger. Skip reprocessing if all necessary files are present.
   - **Function Used:** `check_outputs_folder()`

6. **Process UBE Files**:
   - **Description:** For each UFI logger with UBE files, process and save the data.
   - **Function Used:** `process_ube_file()`

7. **Process CSV Files**:
   - **Description:** For each logger with multiple CSV files, concatenate them, and save the combined data.
   - **Function Used:** `concatenate_and_save_csvs()`

8. **Final Outputs**:
   - **Description:** Ensure all processed data is saved in the outputs folder with appropriate filenames.
   - **Functions Used:** `save_data()`

In [ ]:
# Assuming you have the metadata and dep_db loaded:
datareader = DataReader()
deployment_folder = datareader.check_deployment_folder(dep_db, data_dir)

if deployment_folder:
    datareader.read_files(metadata, save_csv=True, save_parq=True)

In [ ]:
# Optionally look at first notes that have been read in
#datareader.selected_deployment['Time Zone']
#datareader.info['UF-01']
datareader.notes_df[0:5]
#datareader.data['CC-96']
#datareader.data['UF-01']
#datareader.metadata['channelnames']

## Inspect data

In [ ]:
# Load the data_reader object from the pickle file
pkl_path = os.path.join(deployment_folder, 'outputs', 'data.pkl')

with open(pkl_path, 'rb') as file:
    data_pkl = pickle.load(file)

for logger_id, info in data_pkl.info.items():
    sampling_frequency = info.get('datetime_metadata', {}).get('fs', None)
    if sampling_frequency is not None:
        # Format the sampling frequency to 5 significant digits
        print(f"Sampling frequency for {logger_id}: {sampling_frequency} Hz")
    else:
        print(f"No sampling frequency available for {logger_id}")

### Plot data
Make an interactive plot

In [ ]:
# Load color mappings
color_mapping_path = os.path.join(root_dir, 'color_mappings.json')

# Streamlit sidebar for time range selection
imu_logger_to_use = 'CC-96'
ephys_logger_to_use = 'UF-01'

imu_df = data_pkl.data[imu_logger_to_use]
ephys_df = data_pkl.data[ephys_logger_to_use]
overlap_start_time = max(imu_df['datetime'].min(), ephys_df['datetime'].min()).to_pydatetime()
overlap_end_time = min(imu_df['datetime'].max(), ephys_df['datetime'].max()).to_pydatetime()

# Define notes to plot
notes_to_plot = {
    'heartbeat_manual_ok': 'ecg',
    'exhalation_breath': 'depth'
}

# Plotting
fig = plot_tag_data_interactive(data_pkl, imu_channels=['depth', 'accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ', 'magX', 'magY', 'magZ'], 
                                ephys_channels=['ecg'], 
                                imu_logger=imu_logger_to_use, 
                                ephys_logger=ephys_logger_to_use, 
                                time_range=(overlap_start_time, overlap_end_time), 
                                note_annotations=notes_to_plot, 
                                color_mapping_path=color_mapping_path)

fig.show()

In [ ]:
# Plotting again (this takes longer without subplots but allows you to track the time across all plots in a grid)
fig = plot_tag_data_interactive2(data_pkl, imu_channels=['depth', 'accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ', 'magX', 'magY', 'magZ'], 
                                ephys_channels=['ecg'], 
                                imu_logger=imu_logger_to_use, 
                                ephys_logger=ephys_logger_to_use, 
                                time_range=(overlap_start_time, overlap_end_time), 
                                note_annotations=notes_to_plot, 
                                color_mapping_path=color_mapping_path)

fig.show()

In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt

# Function to apply a low-pass filter to extract the static component (gravity)
def low_pass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Function to calculate ODBA
def calculate_odba(accX, accY, accZ, cutoff=0.1, fs=10):
    # Apply low-pass filter to get the static acceleration
    accX_static = low_pass_filter(accX, cutoff, fs)
    accY_static = low_pass_filter(accY, cutoff, fs)
    accZ_static = low_pass_filter(accZ, cutoff, fs)

    # Subtract the static component to get the dynamic acceleration
    accX_dynamic = accX - accX_static
    accY_dynamic = accY - accY_static
    accZ_dynamic = accZ - accZ_static

    # Calculate ODBA
    odba = np.abs(accX_dynamic) + np.abs(accY_dynamic) + np.abs(accZ_dynamic)
    
    return odba

# Example usage with your data
accX = data_pkl.data['CC-96']['accX_adjusted'].values
accY = data_pkl.data['CC-96']['accY_adjusted'].values
accZ = data_pkl.data['CC-96']['accZ_adjusted'].values

odba = calculate_odba(accX, accY, accZ)

data_pkl.data['CC-96']['odba'] = odba

imu_channels_to_plot = ['depth', 'accX', 'accY', 'accZ', 'odba', 'pitch_deg', 'roll_deg', 'heading_deg']
ephys_channels_to_plot = []
imu_logger_to_use = 'CC-96'
ephys_logger_to_use = 'UF-01'

# Get the overlapping time range
imu_df = data_pkl.data[imu_logger_to_use]
ephys_df = data_pkl.data[ephys_logger_to_use]
start_time = max(imu_df['datetime'].min(), ephys_df['datetime'].min()).to_pydatetime()
end_time = min(imu_df['datetime'].max(), ephys_df['datetime'].max()).to_pydatetime()

# Define notes to plot
notes_to_plot = {
    'exhalation_breath': 'depth'
}

plot_tag_data_interactive(data_pkl, imu_channels_to_plot, imu_sampling_rate=1, ephys_channels=ephys_channels_to_plot, 
                          imu_logger=imu_logger_to_use, ephys_logger=ephys_logger_to_use, note_annotations= notes_to_plot,
                          time_range=(start_time, end_time), color_mapping_path=color_mapping_path)

In [ ]:
import os
import trimesh
import numpy as np
from pythreejs import *
from IPython.display import display

# Define the path to the OBJ file
model_path = os.path.join(data_dir, '6_killerWhale_v017_LP.obj')

# Load the OBJ file using Trimesh
mesh = trimesh.load(model_path)

# Extract vertices and faces from the mesh
vertices = mesh.vertices
faces = mesh.faces.astype(np.uint32)  # Ensure the indices are unsigned integers

# Create BufferGeometry for PyThreeJS
geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(vertices, normalized=False),
        'index': BufferAttribute(faces.flatten(), normalized=False)
    }
)

# Create a material
material = MeshLambertMaterial(color='red') # , roughness=0.9, metalness=0.2
key_light = DirectionalLight(color='white', position=[3, 5, 1], intensity=0.75)

# Create the Mesh
mesh = Mesh(geometry=geometry, material=material, position=[0, 0, 200], rotation= [0, math.pi, 0, 'XYZ'])

# Move (translate) the mesh
mesh.position = [0, 0, 200]  # Move the mesh up by 1 unit on the Y-axis

# Rotate the mesh
#mesh.rotation = [0.5, 0.5, 0, 0]  # Rotate the mesh around the X and Y axes (in radians)

# Scale the mesh
mesh.scale = [1.5, 1.5, 1.5]  # Scale the mesh uniformly by 1.5 on all axes

# Create a scene, camera, and renderer
scene = Scene(children=[mesh, AmbientLight(color='#ffffff', intensity=0.5)])
camera = PerspectiveCamera(position=[0, 0, 5], up=[0, 1, 0], children=[key_light])
renderer = Renderer(camera=camera, scene=scene, controls=[OrbitControls(controlling=camera)], width=400, height=400)

# Display the model in the notebook
display(renderer)

ball = Mesh(geometry=SphereGeometry(), 
            material=MeshLambertMaterial(color='red'))


c = PerspectiveCamera(position=[0, 5, 5], up=[0, 1, 0], children=[key_light])

scene = Scene(children=[ball, c, AmbientLight(color='#777777')], background=None)

renderer = Renderer(camera=c, 
                    scene=scene,
                    alpha=True,
                    clearOpacity=0,
                    controls=[OrbitControls(controlling=c)])
display(renderer)

In [ ]:
data_pkl.info['UF-01']['datetime_metadata']['fs']